## VOTER (SoE) information
Load voter information from an OconEco postgres view to get  
SoE voter data that has been geocoded to allow matching to census data 

* This is helpful in comparing registration rates in blockgroups/population groups
* todo: bring in parcels with geolocation matches to census data

In [48]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pandas as pd

host = 'localhost'
port = 5432
db = 'geocoder'
# using a view (vw) on the voters table joined to geolocations so we can get geolocated blockcodes
table_voters = 'vw_voters'

engine = create_engine(f'postgresql://sean:pass1234@{host}:{port}/{db}')
Session = sessionmaker(bind=engine)
session = Session()

# voter_fields = 'select voterid, voter from vw_voters'
# sql = "select * from vw_voters"
sql = """select 
       v.voterid,
       geoid,
       -- SUBSTR(v.geoid, 3,9) as ctbg,
       SUBSTR(v.geoid, 6,7) as ctbg,
       v.voter_name,
       v.white_other,
       v.workforce,
       v.male_other,
       v.affiliation,
       v.vote_history,
       v.house_district,
       v.zip_code,
--        v.geom,
--        v.geom_3857,
--        v.fips_state,
--        v.fips_county,
       v.census_tract,
       v.census_block,
--       'CT:' || v.census_tract || ' bg:' || v.census_block as tract_bg,
       v.birthdate,
       v.yearsold,
       v.sex,
       v.race,
       v.party,
       v.public_email_address,
       v.telephone_number,
       v.residence_address,
       v.formatted_address,
       v.registration_date,
       v."2024_primary",
       v.match_type,
       v.geocoder
from vw_voters v"""
df_voters = pd.read_sql(sql, engine)

In [49]:
print(f"Voters count: {len(df_voters)}")
df_voters.head()

Voters count: 634684


,voterid,geoid,ctbg,voter_name,white_other,workforce,male_other,affiliation,vote_history,house_district,...,race,party,public_email_address,telephone_number,residence_address,formatted_address,registration_date,2024_primary,match_type,geocoder
0,126887522,120310159242019,0159242,"Bakhil, Najah",White,Retired,Female+,DD,Other,17,...,5,Npa,None,None,7232 Bowden Rd,7232 Bowden Rd,05/20/2019,N,Exact,US Census
1,126887535,120310158051000,0158051,"Ebadi, Sulaiman K",White,WorkForce,Female+,DD,2022 General,14,...,5,Npa,None,None,8487 Thor St,8487 Thor St,05/20/2019,N,Exact,US Census
2,126887586,120310137312005,0137312,"Marfut, Sarah M",White,WorkForce,Female+,DD,2022 General,12,...,5,Npa,None,(904)316-3123,8352 Cross Timbers Dr E,8352 Cross Timbers Dr E,05/22/2019,N,Exact,US Census
3,126887593,120310107002023,0107002,"Davis, Kayla S",OTHER,WorkForce,Female+,DD,Other,13,...,3,Npa,None,(904)258-8177,7028 Lincoln Cir W,7028 Lincoln Cir W,05/23/2019,N,Exact,US Census
4,126887608,120310143281009,0143281,"Shami, Sophia T",OTHER,WorkForce,Female+,DD,2020 General,16,...,9,Npa,None,None,11066 Creekview Dr,11066 Creekview Dr,05/20/2019,N,Exact,US Census


### Create user defined function (UDF) to classify age in 3 groups

In [50]:
# Step 1: Categorize age groups
# NOTE: this is focused on voter data, so there should be no 'under18' records 
def categorize_age(age):
    if age < 18:
        return 'under18'
    elif 18 <= age <= 65:
        return '18to65'
    else:
        return 'over65'

In [51]:
# Apply age categorization
df_voters['v_age_group'] = df_voters['yearsold'].apply(categorize_age)
# df_voters['label'] = f"CT:{df_voters['census_tract']} bt:{df_voters['census_block']}"
df_voters['label'] = "CT:" + df_voters['census_tract'].astype(str) + " bg:" + df_voters['census_block'].astype(str)
print(f"Sneak peak of select data, columns: {df_voters.columns}...")
df_voters.filter(items=['voterid', 'label', 'v_age_group', 'yearsold', 'birthdate', 'male_other', 'white_other', 'affiliation', 'workforce', '2024_primary']).head()

Sneak peak of select data, columns: Index(['voterid', 'geoid', 'ctbg', 'voter_name', 'white_other', 'workforce',
       'male_other', 'affiliation', 'vote_history', 'house_district',
       'zip_code', 'census_tract', 'census_block', 'birthdate', 'yearsold',
       'sex', 'race', 'party', 'public_email_address', 'telephone_number',
       'residence_address', 'formatted_address', 'registration_date',
       '2024_primary', 'match_type', 'geocoder', 'v_age_group', 'label'],
      dtype='object')...


,voterid,label,v_age_group,yearsold,birthdate,male_other,white_other,affiliation,workforce,2024_primary
0,126887522,CT:015924 bg:2019,over65,75.0,1949-08-15,Female+,White,DD,Retired,N
1,126887535,CT:015805 bg:1000,18to65,35.0,1988-11-15,Female+,White,DD,WorkForce,N
2,126887586,CT:013731 bg:2005,18to65,23.0,2001-01-17,Female+,White,DD,WorkForce,N
3,126887593,CT:010700 bg:2023,18to65,32.0,1992-07-27,Female+,OTHER,DD,WorkForce,N
4,126887608,CT:014328 bg:1009,18to65,50.0,1974-06-20,Female+,OTHER,DD,WorkForce,N


### Aggregate voter rows
Prepare for pivoting (below) and calculating Quintiles

In [52]:
# simple voter count by ctbg which will then get the results of crosstabs below
df_voters_aggr = df_voters.groupby(['label'], observed=True).agg(
    v_voter_count=('voterid', 'count'),  # use size to count rows
)
# df_voters_aggr

#### explode category columns into counts of value columns

In [53]:
# Crosstab for each column and merge the results
count_race = pd.crosstab(df_voters['label'], df_voters['white_other'])
# count_race            
count_age = pd.crosstab(df_voters['label'], df_voters['v_age_group'])
count_gender = pd.crosstab(df_voters['label'], df_voters['male_other'])
count_affiliation = pd.crosstab(df_voters['label'], df_voters['affiliation'])
count_history = pd.crosstab(df_voters['label'], df_voters['vote_history'])
count_workforce = pd.crosstab(df_voters['label'], df_voters['workforce'])
print(f"Sneak peak of select data, columns: {count_workforce.columns}...")
count_workforce

Sneak peak of select data, columns: Index(['Retired', 'WorkForce'], dtype='object', name='workforce')...


workforce,Retired,WorkForce
label,,
CT:000100 bg:4002,1,0
CT:000100 bg:4025,1,0
CT:000101 bg:1,0,5
CT:000101 bg:1003,5,59
CT:000101 bg:1004,3,13
...,...,...
CT:970400 bg:3032,0,2
CT:970500 bg:3005,0,1
CT:972902 bg:3000,0,1


#### Merge cross-tab exploded dfs into main voter aggregated df
We need the above count dfs in order to do a few quintalizations (below)

In [54]:

# Merge all the crosstab results on the 'label' column
df_pre_aggr_voter = df_voters_aggr.join([count_race, count_age, count_gender, count_affiliation, count_history, count_workforce, ])

# Reset index if needed
df_pre_aggr_voter = df_pre_aggr_voter.reset_index()

df_pre_aggr_voter

,label,v_voter_count,OTHER,White,18to65,over65,Female+,Male,DD,Dem,Rep,2020 General,2022 General,2024 Primary,Ineligible,Other,Retired,WorkForce
0,CT:000100 bg:4002,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0
1,CT:000100 bg:4025,1,0,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0
2,CT:000101 bg:1,5,3,2,5,0,4,1,2,2,1,0,0,0,5,0,0,5
3,CT:000101 bg:1003,64,48,16,60,4,34,30,10,44,10,7,19,13,2,23,5,59
4,CT:000101 bg:1004,16,12,4,14,2,8,8,1,12,3,2,3,7,2,2,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10668,CT:970400 bg:3032,2,0,2,2,0,1,1,0,0,2,2,0,0,0,0,0,2
10669,CT:970500 bg:3005,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,1
10670,CT:972902 bg:3000,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1
10671,CT:975400 bg:2007,3,1,2,3,0,2,1,0,0,3,3,0,0,0,0,0,3


In [55]:
# Alternative approach to pivot & join above
# Using garoupby and get_dummies to explode the race column
# df_race_dummies = pd.get_dummies(df_voters['race'], prefix='race')

# 
# df__dummies = pd.get_dummies(df_voters[''], prefix='')
# df__dummies
# 
# df__dummies = pd.get_dummies(df_voters[''], prefix='')
# df__dummies
# 
# df__dummies = pd.get_dummies(df_voters[''], prefix='')
# df__dummies
# 
# df__dummies = pd.get_dummies(df_voters[''], prefix='')
# df__dummies

In [56]:
# another alternative??
# # df_voters.dropna(subset=['label'], inplace=True)
# df_voters_aggr = df_voters.groupby(['label'], observed=True).agg(
#     v_voter_count=('voterid', 'count'),  # use size to count rows
#     v_age_18to65_count=('v_age_18to65', 'sum'),
#     v_age_over65_count=('v_over65', 'sum'),
#     v_male_count=('v_male', 'sum'),
#     v_female_count=('v_female', 'sum'),
#     v_dem_count=('v_dem', 'sum'),
#     v_rep_count=('v_rep', 'sum'),
#     v_undecided_count=('v_undecided', 'sum'),
#     v_race1_count=('race_1', 'count'),
#     v_race2_count=('race_2', 'count'),
#     v_race3_count=('race_3', 'count'),
# )
# df_voters_aggr

In [57]:
#third alternative??
# # Add boolean columns for easy aggregation
# df_voters['v_under18'] = (df_voters['v_age_group'] == 'under18').astype(int)
# df_voters['v_age_18to65'] = (df_voters['v_age_group'] == '18to65').astype(int)
# df_voters['v_over65'] = (df_voters['v_age_group'] == 'over65').astype(int)
# df_voters['v_male'] = (df_voters['male_other'] == 'Male').astype(int)
# df_voters['v_female'] = (df_voters['male_other'] == 'Female+').astype(int)
# df_voters['v_dem'] = (df_voters['affiliation'] == 'Dem').astype(int)
# df_voters['v_rep'] = (df_voters['affiliation'] == 'Rep').astype(int)
# df_voters['v_undecided'] = (df_voters['affiliation'] == 'DD').astype(int)

In [58]:
df_census_with_quintiles

,GEO_ID,label,c_TotalPopulation,c_over65Pop,c_over65Percent,c_over65Running,c_Q_over65,c_under18pop,c_under18Percent,c_under18Running,c_Q_under18,NAME,c_laborForcePop,c_laborForcePercent,c_laborForceRunning,c_Q_laborForce,c_checksumPop
312,1500000US120310139062,CT:013906 bg:2,756,630,83.3,948651,5th,0,0.0,2602,1st,"Block Group 2, Census Tract 139.06, Duval Coun...",126,0.2,756,1st,756
100,1500000US120310167272,CT:016727 bg:2,1955,1128,57.7,947236,5th,241,12.3,215568,2nd,"Block Group 2, Census Tract 167.27, Duval Coun...",586,0.3,2711,1st,1955
502,1500000US120310025013,CT:002501 bg:3,942,544,57.7,945281,5th,122,13.0,77438,1st,"Block Group 3, Census Tract 25.01, Duval Count...",276,0.3,3653,1st,942
442,1500000US120310144221,CT:014422 bg:1,1359,778,57.2,944339,5th,167,12.3,125791,1st,"Block Group 1, Census Tract 144.22, Duval Coun...",414,0.3,5012,1st,1359
363,1500000US120310144133,CT:014413 bg:3,1920,907,47.2,939247,5th,361,18.8,399654,3rd,"Block Group 3, Census Tract 144.13, Duval Coun...",652,0.3,6932,1st,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,1500000US120310144282,CT:014428 bg:2,1878,46,2.4,44289,1st,101,5.4,52377,1st,"Block Group 2, Census Tract 144.28, Duval Coun...",1731,0.9,942918,5th,1878
37,1500000US120310144152,CT:014415 bg:2,1681,18,1.1,13480,1st,64,3.8,35032,1st,"Block Group 2, Census Tract 144.15, Duval Coun...",1599,1.0,944599,5th,1681
447,1500000US120310144143,CT:014414 bg:3,1155,0,0.0,5372,1st,14,1.2,9206,1st,"Block Group 3, Census Tract 144.14, Duval Coun...",1141,1.0,945754,5th,1155
238,1500000US120310144151,CT:014415 bg:1,2897,0,0.0,4217,1st,104,3.6,57911,1st,"Block Group 1, Census Tract 144.15, Duval Coun...",2793,1.0,948651,5th,2897


In [59]:
df_census_filtered

,GEO_ID,label,c_TotalPopulation,c_over65Pop,c_over65Percent,c_over65Running,c_Q_over65,c_under18pop,c_under18Percent,c_under18Running,c_Q_under18,NAME,c_laborForcePop,c_laborForcePercent,c_laborForceRunning,c_Q_laborForce,c_checksumPop
312,1500000US120310139062,CT:013906 bg:2,756,630,83.3,948651,5th,0,0.0,2602,1st,"Block Group 2, Census Tract 139.06, Duval Coun...",126,0.2,756,1st,756
100,1500000US120310167272,CT:016727 bg:2,1955,1128,57.7,947236,5th,241,12.3,215568,2nd,"Block Group 2, Census Tract 167.27, Duval Coun...",586,0.3,2711,1st,1955
502,1500000US120310025013,CT:002501 bg:3,942,544,57.7,945281,5th,122,13.0,77438,1st,"Block Group 3, Census Tract 25.01, Duval Count...",276,0.3,3653,1st,942
442,1500000US120310144221,CT:014422 bg:1,1359,778,57.2,944339,5th,167,12.3,125791,1st,"Block Group 1, Census Tract 144.22, Duval Coun...",414,0.3,5012,1st,1359
363,1500000US120310144133,CT:014413 bg:3,1920,907,47.2,939247,5th,361,18.8,399654,3rd,"Block Group 3, Census Tract 144.13, Duval Coun...",652,0.3,6932,1st,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,1500000US120310144282,CT:014428 bg:2,1878,46,2.4,44289,1st,101,5.4,52377,1st,"Block Group 2, Census Tract 144.28, Duval Coun...",1731,0.9,942918,5th,1878
37,1500000US120310144152,CT:014415 bg:2,1681,18,1.1,13480,1st,64,3.8,35032,1st,"Block Group 2, Census Tract 144.15, Duval Coun...",1599,1.0,944599,5th,1681
447,1500000US120310144143,CT:014414 bg:3,1155,0,0.0,5372,1st,14,1.2,9206,1st,"Block Group 3, Census Tract 144.14, Duval Coun...",1141,1.0,945754,5th,1155
238,1500000US120310144151,CT:014415 bg:1,2897,0,0.0,4217,1st,104,3.6,57911,1st,"Block Group 1, Census Tract 144.15, Duval Coun...",2793,1.0,948651,5th,2897


In [60]:
df_census_voter = df_pre_aggr_voter.merge(df_census_filtered, on='label', how='outer')
df_census_voter

,label,v_voter_count,OTHER,White,18to65,over65,Female+,Male,DD,Dem,...,c_under18pop,c_under18Percent,c_under18Running,c_Q_under18,NAME,c_laborForcePop,c_laborForcePercent,c_laborForceRunning,c_Q_laborForce,c_checksumPop
0,CT:000100 bg:4002,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CT:000100 bg:4025,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CT:000101 bg:1,5.0,3.0,2.0,5.0,0.0,4.0,1.0,2.0,2.0,...,62.0,8.5,28834.0,1st,"Block Group 1, Census Tract 1.01, Duval County...",511.0,0.7,575842.0,4th,729.0
3,CT:000101 bg:1003,64.0,48.0,16.0,60.0,4.0,34.0,30.0,10.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CT:000101 bg:1004,16.0,12.0,4.0,14.0,2.0,8.0,8.0,1.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10705,CT:970500 bg:3005,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10706,CT:972902 bg:3000,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10707,CT:975400 bg:2007,3.0,1.0,2.0,3.0,0.0,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10708,CT:990000 bg:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,2776.0,1st,"Block Group 0, Census Tract 9900, Duval County...",0.0,NaN,948651.0,5th,0.0


### After aggrgegating the raw voter data
pivot several fields to get counts of values 

### Sort Voters and calculate Quartiles

In [ ]:
# df_voters.sort_values(by=)

In [ ]:
print(f"New columns: {df_voters.columns}")
print(f"row count: {len(df_voters)}")
df_voters

In [ ]:
print(f"census fields (as a reminder/reference)...")
df_census_filtered.columns

In [ ]:
df_voter_census = pd.merge(df_voters, df_census_with_quintiles, on='ctbg', how='left')
print(f"df_voter_census shape: {df_voter_census.shape}")

In [ ]:
fname = "voter-census_details.csv"
df_voter_census.to_csv(fname, index=False)
print(f"wrote file: voter-census_details.csv, field count:({len(df_voter_census.columns)})")

In [ ]:
df_voter_census

In [ ]:
# Now perform the groupby and aggregation
df_voters_aggr = df_voter_census.groupby(['label', 'c_TotalPopulation',
                                   'c_Q_over65', 'c_over65Pop', 'c_over65Percent',
                                   'c_Q_laborForce', 'c_laborForcePop', 'c_laborForcePercent', 
                                   'c_Q_under18', 'c_under18pop', 'c_under18Percent',
                                   'c_checksumPop'
                                          ], observed=True).agg(
    v_voter_count=('voterid', 'count'),  # use size to count rows
    v_under18_count=('v_under18', 'sum'),
    v_age_18to65_count=('v_age_18to65', 'sum'),
    v_over65_count=('v_over65', 'sum'),
    v_male_count=('v_male', 'sum'),
    v_female_count=('v_female', 'sum'),
    v_dem_count=('v_dem', 'sum'),
    v_rep_count=('v_rep', 'sum'),
    v_undecided_count=('v_undecided', 'sum'),

)
df_voters_aggr

In [ ]:
fname = "voter-census_summary.csv"
df_voter_census_aggr = pd.merge(df_voters_aggr, df_census_with_quintiles, on='ctbg', how='left')
df_voter_census_aggr.to_csv(fname, index=False)
print(f"Wrote file: {fname}")

In [ ]:
df_voter_census_aggr

## Aggregate the combined SoE(voter) and Augmented Census data


In [ ]:
df_voter_census_aggr

In [ ]:

# Show the df_voter_aggr DataFrame
# import ace_tools as tools;
# tools.display_dataframe_to_user(name="df_voter_aggr Voter Data", dataframe=df_voter_aggr)


In [ ]:
# df_minmax = df_subset.groupby('Quintile')['Running Sum Population'].agg(['min', 'max', 'count'])
# Calculate the difference between 'max' and 'min' and store it in a new column called 'difference'
# df_minmax['Pop Range Difference'] = df_minmax['max'] - df_minmax['min']
# print("Show the min/max and difference in population size for each quintile (value ranges should be roughly equivalent)")
# df_minmax = df_minmax.rename(columns={'min': 'Min under18Percent', 'max': 'Max under18Percent', 'count': 'Quintile Row Count'})
# df_minmax

### TODO & next steps

* Sanity check the calculations and process
* Add in percentages on quintile-boundaries (q-boundaries)
* smooth the q-boundaries to whole number percents
